In [ ]:
from google.colab import drive
from google.colab import files
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Step 1: Mount Google Drive and Load the Model
drive.mount('/content/drive')

# Path to the saved model in Google Drive
model_path = '/content/drive/My Drive/Research/Models/sinhala_character_model_v4.h5'

# Load the saved model
model = load_model(model_path)

# Step 2: Sinhala Unicode Mapping
sinhala_unicode_mapping = {
    1: "අ", 2: "ආ", 3: "ඇ", 4: "ඈ", 5: "ඉ", 6: "ඊ", 7: "උ", 8: "එ", 9: "ඒ", 10: "ඔ",
    11: "ඕ", 12: "ක", 13: "කා", 14: "කැ", 15: "කෑ", 16: "කි", 17: "කී", 18: "කු", 19: "කූ", 20: "ක්",
    21: "කෝ", 22: "ක්‍ර", 23: "ක්‍රි", 24: "ක්‍රී", 25: "ග", 26: "ගා", 27: "ගැ", 28: "ගෑ", 29: "ගි", 30: "ගී",
    31: "ගු", 32: "ගූ", 33: "ග්", 34: "ගෝ", 35: "ග්‍ර", 36: "ග්‍රි", 37: "ග්‍රී", 38: "ච", 39: "චා", 40: "චැ",
    41: "චෑ", 42: "චි", 43: "චී", 44: "චු", 45: "චූ", 46: "ච්", 47: "චෝ", 48: "ච්‍ර", 49: "ච්‍ර්", 50: "ච්‍රී",
    51: "ජ", 52: "ජා", 53: "ජැ", 54: "ජෑ", 55: "ජි", 56: "ජී", 57: "ජු", 58: "ජූ", 59: "ජ්", 60: "ජෝ",
    61: "ජ්‍ර", 62: "ජ්‍රි", 63: "ජ්‍රී", 64: "ට", 65: "ටා", 66: "ටැ", 67: "ටෑ", 68: "ටි", 69: "ටී", 70: "ටු",
    71: "ටූ", 72: "ට්", 73: "ටෝ", 74: "ට්‍ර", 75: "ට්‍ර්", 76: "ට්‍රි", 77: "ඩ", 78: "ඩා", 79: "ඩැ", 80: "ඩෑ",
    81: "ඩි", 82: "ඩී", 83: "ඩු", 84: "ඩූ", 85: "ඩ්", 86: "ඩෝ", 87: "ඩ්‍ර", 88: "ඩ්‍ර්", 89: "ඩ්‍රි", 90: "ණ",
    91: "ණා", 92: "ණි", 93: "ත", 94: "තා", 95: "ති", 96: "තී", 97: "තු", 98: "තූ", 99: "ත්", 100: "තෝ",
    101: "ත්‍ර", 102: "ත්‍රා", 103: "ත්‍රි", 104: "ත්‍රී", 105: "ද", 106: "දා", 107: "දැ", 108: "දෑ", 109: "දි", 110: "දී",
    111: "දු", 112: "දූ", 113: "ද්", 114: "දෝ", 115: "ද්‍ර", 116: "ද්‍රෝ", 117: "ද්‍රා", 118: "ද්‍රි", 119: "ද්‍රී", 120: "න",
    121: "නා", 122: "නැ", 123: "නෑ", 124: "නි", 125: "නී", 126: "නු", 127: "නූ", 128: "න්", 129: "නෝ", 130: "න්‍ර",
    131: "න්‍රා", 132: "න්‍රි", 133: "න්‍රී", 134: "ප", 135: "පා", 136: "පැ", 137: "පෑ", 138: "පි", 139: "පී", 140: "පු",
    141: "පූ", 142: "ප්", 143: "ප්‍රෝ", 144: "පෝ", 145: "ප්‍ර", 146: "ප්‍රා", 147: "ප්‍රි", 148: "ප්‍රී", 149: "බ", 150: "බා",
    151: "බැ", 152: "බෑ", 153: "බි", 154: "බී", 155: "බු", 156: "බූ", 157: "බ්", 158: "බ්‍රෝ", 159: "බ්‍ර", 160: "බ්‍රා",
    161: "බ්‍රි", 162: "බ්‍රී", 163: "බ්‍රෝ", 164: "ම", 165: "මා", 166: "මැ", 167: "මෑ", 168: "මි", 169: "මී", 170: "මු",
    171: "මූ", 172: "ම්", 173: "මෝ", 174: "ම්‍ර", 175: "ම්‍රා", 176: "ම්‍රි", 177: "ම්‍රී", 178: "ම්‍රෝ", 179: "ය", 180: "යා",
    181: "යැ", 182: "යෑ", 183: "යි", 184: "යී", 185: "යු", 186: "යූ", 187: "ෝ", 188: "ය්", 189: "යෝ", 190: "ර",
    191: "රා", 192: "රැ", 193: "රෑ", 194: "රු", 195: "රූ", 196: "රි", 197: "රී", 198: "ල", 199: "ලා", 200: "ලැ",
    201: "ලෑ", 202: "ලි", 203: "ලී", 204: "ලු", 205: "ලූ", 206: "ල්", 207: "ලෝ", 208: "ව", 209: "වා", 210: "වැ",
    211: "වෑ", 212: "වි", 213: "වී", 214: "වු", 215: "වූ", 216: "ව්", 217: "වෝ", 218: "ව්‍ර", 219: "ව්‍රා", 220: "ව්‍රැ",
    221: "ව්‍රෑ", 222: "ව්‍රෝ", 223: "ශ", 224: "ශා", 225: "ශැ", 226: "ශෑ", 227: "ශි", 228: "ශී", 229: "ශු", 230: "ශූ",
    231: "ශ්", 232: "ශෝ", 233: "ශ්‍ර", 234: "ශ්‍රා", 235: "ශ්‍රැ", 236: "ශ්‍රෑ", 237: "ශ්‍රි", 238: "ශ්‍රී", 239: "ශ්‍රෝ", 240: "ෂ",
    241: "ෂා", 242: "ෂැ", 243: "ෂෑ", 244: "ෂි", 245: "ෂී", 246: "ෂු", 247: "ෂූ", 248: "ෂ්", 249: "ෂෝ", 250: "ස",
    251: "සා", 252: "සැ", 253: "සෑ", 254: "සි", 255: "සී", 256: "සු", 257: "සූ", 258: "සෝ", 259: "ස්‍ර", 260: "ස්‍රා",
    261: "ස්‍රි", 262: "ස්‍රී", 263: "ස්", 264: "හ", 265: "හා", 266: "හැ", 267: "හෑ", 268: "හි", 269: "හී", 270: "හු",
    271: "හූ", 272: "හ්", 273: "හෝ", 274: "ළ", 275: "ළා", 276: "ළැ", 277: "ළෑ", 278: "ළි", 279: "ළී", 280: "ළු",
    281: "ළූ", 282: "ෆ", 283: "ෆා", 284: "ෆැ", 285: "ෆෑ", 286: "ෆි", 287: "ෆී", 288: "ෆු", 289: "ෆූ", 290: "ෆ්‍ර",
    291: "ෆ්‍රි", 292: "ෆ්‍රී", 293: "ෆ්‍රැ", 294: "ෆ්‍රෑ", 295: "ෆ්", 296: "ෆෝ", 297: "ක්‍රා", 298: "ක්‍රැ", 299: "ක්‍රෑ", 300: "ක්‍රෝ",
    301: "ග්‍රෝ", 302: "ඛ", 303: "ඛා", 304: "ඛි", 305: "ඛී", 306: "ඛ්", 307: "ඝ", 308: "ඝා", 309: "ඝැ", 310: "ඝෑ",
    311: "ඝි", 312: "ඝී", 313: "ඝු", 314: "ඝූ", 315: "ඝෝ", 316: "ඝ්", 317: "ඝ්‍ර", 318: "ඝ්‍රා", 319: "ඝ්‍රි", 320: "ඝ්‍රී",
    321: "ඳ", 322: "ඳා", 323: "ඳැ", 324: "ඳෑ", 325: "ෑ", 326: "ඳි", 327: "ඳී", 328: "ඳු", 329: "ඳූ", 330: "ඳෝ",
    331: "ඳ්", 332: "ඟ", 333: "ඟා", 334: "ඟැ", 335: "ඟෑ", 336: "ඟි", 337: "ඟී", 338: "ඟු", 339: "ඟූ", 340: "ඟෝ",
    341: "ඟ්", 342: "ඬ", 343: "ැ", 344: "ඬා", 345: "ඬැ", 346: "ඬෑ", 347: "ඬි", 348: "ඬී", 349: "ඬු", 350: "ඬූ",
    351: "ඬෝ", 352: "ඬ්", 353: "ඹ", 354: "ඹා", 355: "ඹැ", 356: "ඹෑ", 357: "ඹි", 358: "ඹී", 359: "ඹු", 360: "ඹූ",
    361: "ඹෝ", 362: "ඹ්", 363: "භ", 364: "භා", 365: "භැ", 366: "භෑ", 367: "භි", 368: "භී", 369: "භු", 370: "භූ",
    371: "භෝ", 372: "භ්", 373: "ධ", 374: "ධා", 375: "ධැ", 376: "ධෑ", 377: "ධි", 378: "ධී", 379: "ධු", 380: "ධූ",
    381: "ධෝ", 382: "ධ්", 383: "ඨ", 384: "ඨා", 385: "ඨැ", 386: "ඨි", 387: "ඨී", 388: "ඨු", 389: "ඨූ", 390: "ඨ්",
    391: "ඪ", 392: "ඪා", 393: "ඪි", 394: "ඪෝ", 395: "ඵ", 396: "ඵා", 397: "ඵු", 398: "ඵි", 399: "ඵෝ", 400: "ඵ්",
    401: "ථ", 402: "ථා", 403: "ථැ", 404: "ථ්", 405: "ා", 406: "ෟ", 407: "ණැ", 408: "ණෑ", 409: "ෘ", 410: "ණී",
    411: "ණු", 412: "ණූ", 413: "ණෝ", 414: "ණ්", 415: "ඥ", 416: "ඥා", 417: "ඥෝ", 418: "ඤ", 419: "ඤා", 420: "ඤු",
    421: "ඤෝ", 422: "ඤ්", 423: "ඣ", 424: "ඣා", 425: "ඣු", 426: "ඣෝ", 427: "ඣ්", 428: "ඦ", 429: "ඦා", 430: "ඦැ",
    431: "ඦෑ", 432: "ඦි", 433: "ඦු", 434: "ඦූ", 435: "ඦෝ	", 436: "ඦ", 437: "ඡ", 438: "ඡා", 439: "ඡැ", 440: "ඡෑ",
    441: "ඡි", 442: "ඡේ", 443: "තැ", 444: "තෑ", 445: "ත්‍රැ", 446: "ත්‍රෑ", 447: "ත්‍රෝ", 448: "ළු", 449: "ෲ", 450: "",
    451: "ෛ", 452: "ෙ", 453: "‍", 454: ""
}

# Step 3: File Upload Method
def upload_and_predict():
    """
    Upload an image using file explorer and predict the Sinhala character.
    """
    # Upload file
    uploaded = files.upload()  # Opens file explorer
    for filename in uploaded.keys():
        print(f'Uploaded file: {filename}')

        # Preprocess the uploaded image
        img = preprocess_image(filename)

        # Predict the character
        predictions = model.predict(img)
        predicted_class = np.argmax(predictions)
        predicted_character = sinhala_unicode_mapping.get(predicted_class + 1, "Unknown")

        # Output the result
        print(f'Predicted Character: {predicted_character}')

# Step 4: Preprocessing
def preprocess_image(image_path):
    """
    Preprocesses an image to match the model's input shape.
    - Loads the image in grayscale.
    - Resizes to 80x80.
    - Normalizes pixel values to [0, 1].
    - Expands dimensions to match model input (1, 80, 80, 1).
    """
    img = image.load_img(image_path, color_mode='grayscale', target_size=(80, 80))
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Step 5: Call the upload_and_predict Function
upload_and_predict()
